In [2]:
import quimb as qu
import quimb.tensor as qtn

# Define the system size
N = 10  # Number of spins

# Define the Hamiltonian
# For instance, a 1D Heisenberg model
ham_terms = []
for i in range(N - 1):
    ham_terms += [
        qu.gen.operators.sigmax(i) & qu.operators.sigmax(i + 1),
        qu.gen.operators.sigmay(i) & qu.operators.sigmay(i + 1),
        qu.gen.operators.sigmaz(i) & qu.operators.sigmaz(i + 1),
    ]
H = qu.hamiltonian(ham_terms, dims=[2] * N)

# Convert the Hamiltonian to a Matrix Product Operator (MPO)
H_mpo = qtn.MPO_ham_1d(H)

# Initialize a random MPS
psi = qtn.MPS_rand_state(N, bond_dim=2)

# Perform DMRG
energy, psi_opt = qtn.dmrg(H_mpo, psi, bond_dims=[10, 20, 100], cutoffs=1e-10)

# Output the ground state energy
print("Ground state energy: ", energy)


AttributeError: module 'quimb.gen.operators' has no attribute 'sigmax'

In [6]:
import numpy as np
from quimb.core import (qarray, make_immutable, get_thread_pool,
                    par_reduce, isreal, qu, eye, kron, ikron)
n = 10
j = 1
b = 0
parallel = False
ownership = None
cyclic = False
dims = (2,) * n

op_kws = {'sparse': True, 'stype': 'coo'}
ikron_kws = {'sparse': True, 'stype': 'coo',
             'coo_build': True, 'ownership': ownership}

sxyz = [spin_operator(i, **op_kws) for i in 'xyz']

coosj1 = np.array([(i, i + 1) for i in range(n)])
coosj2 = np.array([(i, i + 2) for i in range(n)])
if cyclic:
    coosj1, coosj2 = coosj1 % n, coosj2 % n
else:
    coosj1 = coosj1[np.all(coosj1 < n, axis=1)]
    coosj2 = coosj2[np.all(coosj2 < n, axis=1)]

def j1_terms():
    for coo in coosj1:
        if abs(coo[1] - coo[0]) == 1:  # can sum then tensor (faster)
            yield ikron(sum(op & op for op in sxyz),
                        dims, coo, **ikron_kws)
        else:  # tensor then sum (slower)
            yield sum(ikron(op, dims, coo, **ikron_kws) for op in sxyz)


ham = j * sum(j1_terms()) 

In [8]:
print(ham)

  (0, 0)	(2.25+0j)
  (1, 1)	(1.75+0j)
  (1, 2)	(0.5+0j)
  (2, 1)	(0.5+0j)
  (2, 2)	(1.25+0j)
  (2, 4)	(0.5+0j)
  (3, 3)	(1.75+0j)
  (3, 5)	(0.5+0j)
  (4, 2)	(0.5+0j)
  (4, 4)	(1.25+0j)
  (4, 8)	(0.5+0j)
  (5, 3)	(0.5+0j)
  (5, 5)	(0.75+0j)
  (5, 6)	(0.5+0j)
  (5, 9)	(0.5+0j)
  (6, 5)	(0.5+0j)
  (6, 6)	(1.25+0j)
  (6, 10)	(0.5+0j)
  (7, 7)	(1.75+0j)
  (7, 11)	(0.5+0j)
  (8, 4)	(0.5+0j)
  (8, 8)	(1.25+0j)
  (8, 16)	(0.5+0j)
  (9, 5)	(0.5+0j)
  (9, 9)	(0.75+0j)
  :	:
  (1014, 1014)	(0.75+0j)
  (1014, 1018)	(0.5+0j)
  (1015, 1007)	(0.5+0j)
  (1015, 1015)	(1.25+0j)
  (1015, 1019)	(0.5+0j)
  (1016, 1012)	(0.5+0j)
  (1016, 1016)	(1.75+0j)
  (1017, 1013)	(0.5+0j)
  (1017, 1017)	(1.25+0j)
  (1017, 1018)	(0.5+0j)
  (1018, 1014)	(0.5+0j)
  (1018, 1017)	(0.5+0j)
  (1018, 1018)	(0.75+0j)
  (1018, 1020)	(0.5+0j)
  (1019, 1015)	(0.5+0j)
  (1019, 1019)	(1.25+0j)
  (1019, 1021)	(0.5+0j)
  (1020, 1018)	(0.5+0j)
  (1020, 1020)	(1.75+0j)
  (1021, 1019)	(0.5+0j)
  (1021, 1021)	(1.25+0j)
  (1021, 1022)	(0.5

In [5]:
def spin_operator(label, S=1 / 2, **kwargs):
    """Generate a general spin-operator.

    Parameters
    ----------
    label : str
        The type of operator, can be one of six options:

            - ``{'x', 'X'}``, x-spin operator.
            - ``{'y', 'Y'}``, y-spin operator.
            - ``{'z', 'Z'}``, z-spin operator.
            - ``{'+', 'p'}``, Raising operator.
            - ``{'-', 'm'}``, Lowering operator.
            - ``{'i', 'I'}``, identity operator.

    S : float, optional
        The spin of particle to act on, default to spin-1/2.
    kwargs
        Passed to :func:`quimbify`.

    Returns
    -------
    S : immutable operator
        The spin operator.

    See Also
    --------
    pauli

    Examples
    --------
    >>> spin_operator('x')
    qarray([[0. +0.j, 0.5+0.j],
            [0.5+0.j, 0. +0.j]])

    >>> qu.spin_operator('+', S=1)
    qarray([[0.        +0.j, 1.41421356+0.j, 0.        +0.j],
            [0.        +0.j, 0.        +0.j, 1.41421356+0.j],
            [0.        +0.j, 0.        +0.j, 0.        +0.j]])

    >>> qu.spin_operator('Y', sparse=True)
    <2x2 sparse matrix of type '<class 'numpy.complex128'>'
        with 2 stored elements in Compressed Sparse Row format>

    """

    D = int(2 * S + 1)

    op = np.zeros((D, D), dtype=complex)
    ms = np.linspace(S, -S, D)

    label = label.lower()

    if label in {'x', 'y'}:
        for i in range(D - 1):
            c = 0.5 * (S * (S + 1) - (ms[i] * ms[i + 1]))**0.5
            op[i, i + 1] = -1.0j * c if (label == 'y') else c
            op[i + 1, i] = 1.0j * c if (label == 'y') else c

    elif label == 'z':
        for i in range(D):
            op[i, i] = ms[i]

    elif label in {'+', 'p', '-', 'm'}:
        for i in range(D - 1):
            c = (S * (S + 1) - (ms[i] * ms[i + 1]))**0.5
            if label in {'+', 'p'}:
                op[i, i + 1] = c
            else:
                op[i + 1, i] = c

    elif label in {'i', 'I'}:
        np.fill_diagonal(op, 1.0)

    else:
        raise ValueError(f"Label '{label}'' not understood, should be one of "
                         "``['X', 'Y', 'Z', '+', '-', 'I']``.")

    op = qu(np.real_if_close(op), **kwargs)

    make_immutable(op)
    return op